In [ ]:
#default_exp learner

In [ ]:
#export
from detectrino.basics import *

# Learner
> Core Learner

In [ ]:
#export
# Dsets is only train dataset for now
# TODO: Loading datset twice: Once for getting the len, other by detectron2 internals
# TODO: How to correctly handle max_iter and lr_schedule?
class DetLearner:
    def __init__(self, dset, mcfg, pretrained=True):
        self.dset = dset
        self.dset_len = self._dset_len()
        cfg = get_cfg()
        cfg.merge_from_file(model_zoo.get_config_file(mcfg.mfile))
        cfg = mergedicts(cfg, mcfg.to_cfg())
        cfg.DATASETS.TRAIN,cfg.DATASETS.TEST = [dset],[]
        if pretrained: cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(mcfg.mfile)
        os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
        self.cfg = cfg
        
    def fit(self, n_epoch, lr, bs=None):
        cfg = self.cfg
        cfg.SOLVER.BASE_LR = lr
        cfg.SOLVER.IMS_PER_BATCH = bs
        trainer = DefaultTrainer(cfg)
        trainer.resume_or_load()
        trainer.max_iter = cfg.SOLVER.MAX_ITER = trainer.start_iter + int(n_epoch*(self.dset_len/bs))
        trainer.train()
        cfg.MODEL.WEIGHTS = str(Path(cfg.OUTPUT_DIR)/'model_final.pth')
        
    def load(self, name):
        cfg.MODEL.WEIGHTS = str(Path(cfg.OUTPUT_DIR)/name)
        trainer.resume_or_load()
        
    def _dset_len(self): 
#         return len(self.trainer.data_loader.dataset.dataset._dataset._addr)
        return len(DatasetCatalog.get(self.dset))

In [ ]:
# TODO: Simple test
# mfile = ModelCatalog.COCO_InstanceSegmentation__mask_rcnn_R_50_FPN_3x
# learn = DetLearner('voc_2007_val', RCNNCfg(mfile, 12, 128))

## Export -

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_data.core.ipynb.
Converted 02_data.config.ipynb.
Converted 05_model.external.ipynb.
Converted 06_model.config.ipynb.
Converted 08_learner.ipynb.
Converted index.ipynb.
